In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
X_train = pd.read_csv("split_data/train_features_preprocessed.csv")
y_train = pd.read_csv("split_data/train_target_preprocessed.csv")

X_train.shape

In [ ]:
X_val = pd.read_csv("split_data/val_features_preprocessed.csv")
y_val = pd.read_csv("split_data/val_target_preprocessed.csv")

# Function to calculate Winkler Score for predicted intervals

https://www.kaggle.com/datasets/carlmcbrideellis/winkler-interval-score-metric

In [ ]:
def WIS_and_coverage(y_true, lower, upper, alpha):
        
        if np.isnan(lower)  == True: 
            raise ParticipantVisibleError("lower interval value contains NaN value(s)")
        if np.isinf(lower)  == True: 
            raise ParticipantVisibleError("lower interval value contains inf values(s)")
        if np.isnan(upper)  == True: 
            raise ParticipantVisibleError("upper interval value contains NaN value(s)")
        if np.isinf(upper)  == True: 
            raise ParticipantVisibleError("upper interval value contains inf values(s)")
        # These should not occur in a competition setting
        if np.isnan(y_true) == True:
            raise ParticipantVisibleError("y_true contains NaN value(s)")
        if np.isinf(y_true) == True: 
            raise ParticipantVisibleError("y_true contains inf values(s)")
        
        # WIS for a single interval
        score = np.abs(upper - lower)
        if y_true < np.minimum(upper, lower):
            score += ((2/alpha) * (np.minimum(upper, lower) - y_true))
        if y_true > np.maximum(upper, lower):
            score += ((2/alpha) * (y_true - np.maximum(upper, lower)))
        # coverage for one single row
        coverage  = 1
        if (y_true < np.minimum(upper, lower)) or (y_true > np.maximum(upper, lower)):
            coverage = 0
        return score, coverage

# vectorize the function
v_WIS_and_coverage = np.vectorize(WIS_and_coverage)

In [ ]:
def score(y_true, lower, upper, alpha):
        
        y_true = y_true.astype(float)
        lower  = lower.astype(float)
        upper  = upper.astype(float)
        
        WIS_score,coverage = v_WIS_and_coverage(y_true, lower, upper, alpha)
        MWIS     = np.mean(WIS_score)
        coverage = coverage.sum() / coverage.shape[0]
        
        MWIS      = float(MWIS)
        coverage  = float(coverage)
        
        return MWIS, coverage

# Gradient boosting regressor

https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-quantile-py

In [ ]:
y_train = np.ravel(y_train)

## Submission 1 to leaderboard

In [ ]:
gbr = GradientBoostingRegressor(loss="absolute_error", verbose = 50)

# hyperparameters grid
param_grid = dict(
    learning_rate=[0.05, 0.1, 0.2],
    max_depth=[5, 10, 15, 20],
    min_samples_leaf=[1, 5, 10, 20, 25],
    min_samples_split=[20, 30, 50, 60, 70],
)

# perform grid search
grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',  
    cv=5,                                      
    n_jobs=-1                           
)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
all_models = {}

common_params = dict(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=20,
    min_samples_split=60,
)

# for point prediction

gbr_mae = GradientBoostingRegressor(loss="absolute_error", **common_params)
all_models["mae"] = gbr_mae.fit(X_train, y_train)

# for interval prediction

for alpha in [0.05, 0.5, 0.95]:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
    all_models["q %1.2f" % alpha] = gbr.fit(X_train, y_train)

In [ ]:
y_pred = all_models["mae"].predict(X_val)
y_lower = all_models["q 0.05"].predict(X_val)
y_upper = all_models["q 0.95"].predict(X_val)
y_med = all_models["q 0.50"].predict(X_val)

In [ ]:
predictions = pd.DataFrame()
predictions['y_true'] = y_calib
predictions["point prediction"] = y_pred
predictions["med"] = y_med
predictions["lower"] = y_lower
predictions["upper"] = y_upper
predictions["midpoint"] = (y_upper+y_lower)/2

predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])
predictions["abs_error_med"] = abs(predictions["med"] - predictions["y_true"])
predictions["abs_error_mid"] = abs(predictions["midpoint"] - predictions["y_true"])

predictions

In [ ]:
MWIS, coverage = score(predictions["y_true"], predictions["lower"], predictions["upper"], alpha = .20)

MWIS

In [ ]:
predictions["abs_error"].mean()

In [ ]:
predictions["abs_error_med"].mean()

In [ ]:
# check cases where point prediction does not fall in the predicted interval

invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
# for invalid cases, replace point prediction by midpoint of predicted interval

predictions['point prediction'] = np.where(
    (predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']),
    predictions['point prediction'],
    predictions['midpoint']
)
predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])

In [ ]:
MWIS, coverage = score(predictions["y_true"], predictions["lower"], predictions["upper"], alpha = .20)

MWIS

In [ ]:
with open('models/gbr_sklearn_models.pkl', 'wb') as file:
    pickle.dump(all_models, file)

## Submission 2 to leaderboard

With lat/lon instead of province

In [ ]:
X_train_2 = pd.read_csv("split_data/train_features_preprocessed_2.csv")
y_train_2 = pd.read_csv("split_data/train_target_preprocessed_2.csv")

In [ ]:
X_val_2 = pd.read_csv("split_data/val_features_preprocessed_2.csv")
y_val_2 = pd.read_csv("split_data/val_target_preprocessed_2.csv")

In [ ]:
y_train_2 = np.ravel(y_train_2)

In [ ]:
gbr = GradientBoostingRegressor(loss="absolute_error", verbose = 50)

# hyperparameters grid
param_grid = dict(
    learning_rate=[0.05, 0.1, 0.2],
    max_depth=[5, 10, 15, 20, 30, 40],
    min_samples_leaf=[10, 20, 25, 30, 40],
    min_samples_split=[20, 30, 50, 60, 70],
)

# perform Grid Search
grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error', 
    cv=5,                                             
    n_jobs=-1                         
)

grid_search.fit(X_train_2, y_train_2)

In [ ]:
grid_search.best_params_

In [ ]:
all_models = {}

common_params = dict(
    learning_rate=0.1,
    n_estimators=200,
    max_depth=30,
    min_samples_leaf=40,
    min_samples_split=20,
)

# for point prediction

gbr_mae = GradientBoostingRegressor(loss="absolute_error", **common_params)
all_models["mae"] = gbr_mae.fit(X_train_2, y_train_2)

# for interval prediction

for alpha in [0.05, 0.5, 0.95]:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
    all_models["q %1.2f" % alpha] = gbr.fit(X_train_2, y_train_2)

In [ ]:
y_pred = all_models["mae"].predict(X_val_2)
y_lower = all_models["q 0.05"].predict(X_val_2)
y_upper = all_models["q 0.95"].predict(X_val_2)
y_med = all_models["q 0.50"].predict(X_val_2)

In [ ]:
predictions = pd.DataFrame()
predictions['y_true'] = y_val_2
predictions["point prediction"] = y_pred
predictions["med"] = y_med
predictions["lower"] = y_lower
predictions["upper"] = y_upper
predictions["midpoint"] = (y_upper+y_lower)/2

predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])
predictions["abs_error_med"] = abs(predictions["med"] - predictions["y_true"])
predictions["abs_error_mid"] = abs(predictions["midpoint"] - predictions["y_true"])

predictions

In [ ]:
invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
predictions['point prediction'] = np.where(
    (predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']),
    predictions['point prediction'],
    predictions['midpoint']
)
predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])

In [ ]:
invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
MWIS, coverage = score(predictions["y_true"], predictions["lower"], predictions["upper"], alpha = .20)

MWIS

In [ ]:
predictions["abs_error"].mean()

In [ ]:
with open('models/gbr_sklearn_models_2.pkl', 'wb') as file:
    pickle.dump(all_models, file)

In [ ]:
importances = all_models["q 0.50"].feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train_2.shape[1]), importances[indices])
plt.xticks(range(X_train_2.shape[1]), X_train_2.columns[indices], rotation=90)
plt.tight_layout()
plt.show()

## Feature importance submission 2

In [ ]:
importances = all_models["mae"].feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train_2.shape[1]), importances[indices])
plt.xticks(range(X_train_2.shape[1]), X_train_2.columns[indices], rotation=90)
plt.tight_layout()
plt.show()

## Attempt 3

Not uploaded because worse results than attempt 2

- Area is imputed taking number of bedrooms into account in addition to subtype
- A few cases with latitudes and longitudes outside of Belgium are removed --> improved quality of imputed (mean) lat and lon
- Statbel data no longer used (external source that needs to be updated): take median price per m2 per type (is appartment Y/N) per zipcode from training data
- Flag price drop added

In [ ]:
X_train_3 = pd.read_csv("split_data/train_features_preprocessed_3.csv")
y_train_3 = pd.read_csv("split_data/train_target_preprocessed_3.csv")

y_train_3 = np.ravel(y_train_3)

In [ ]:
gbr = GradientBoostingRegressor(loss="absolute_error", verbose = 50)

param_grid = {
    'max_depth': [15, 20, 25, 30, 35],
    'min_samples_split': [10, 15, 20, 25],
    'min_samples_leaf': [5, 10, 15, 20],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 150]
}

grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error', 
    cv=5,                                     
    n_jobs=-1                           
)

grid_search.fit(X_train_3, y_train_3)

In [ ]:
grid_search.best_params_

In [ ]:
all_models = {}

common_params = dict(
    learning_rate=0.05,
    n_estimators=150,
    max_depth=35,
    min_samples_leaf=20,
    min_samples_split=15,
)

gbr_mae = GradientBoostingRegressor(loss="absolute_error", **common_params)
all_models["mae"] = gbr_mae.fit(X_train_3, y_train_3)

for alpha in [0.05, 0.5, 0.95]:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
    all_models["q %1.2f" % alpha] = gbr.fit(X_train_3, y_train_3)

In [ ]:
X_valid_3 = pd.read_csv("split_data/val_features_preprocessed_3.csv")
y_valid_3 = pd.read_csv("split_data/val_target_preprocessed_3.csv")

In [ ]:
y_pred = all_models["mae"].predict(X_valid_3)
y_lower = all_models["q 0.05"].predict(X_valid_3)
y_upper = all_models["q 0.95"].predict(X_valid_3)
y_med = all_models["q 0.50"].predict(X_valid_3)

In [ ]:
predictions = pd.DataFrame()
predictions['y_true'] = y_valid_3
predictions["point prediction"] = y_pred
predictions["med"] = y_med
predictions["lower"] = y_lower
predictions["upper"] = y_upper
predictions["midpoint"] = (y_upper+y_lower)/2

predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])
predictions["abs_error_med"] = abs(predictions["med"] - predictions["y_true"])
predictions["abs_error_mid"] = abs(predictions["midpoint"] - predictions["y_true"])

predictions

In [ ]:
invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
predictions['point prediction'] = np.where(
    (predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']),
    predictions['point prediction'],
    predictions['midpoint']
)
predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])

In [ ]:
invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
MWIS, coverage = score(predictions["y_true"], predictions["lower"], predictions["upper"], alpha = .20)

MWIS

In [ ]:
predictions["abs_error"].mean()

## Check worst predictions attempt 3

Worst predictions on most expensive houses

In [ ]:
predictions.sort_values("abs_error", ascending = False).head(30)

## Attempt 3b

Changes applied in attempt 3, but
- Use Statbel data again instead of median prices calculated based on training data
- Median price per area: calculated per group of first 3 zipcode digits (more granular) instead of first 2
- Flag indicating if last 2 (but not 3) or last 3 digits are 0 - indication of bigger cities?

Last two changes are attempts to predict higher prices better (regional differences related to price)

In [ ]:
X_train_3b = pd.read_csv("split_data/train_features_preprocessed_3b.csv")
y_train_3b = pd.read_csv("split_data/train_target_preprocessed_3b.csv")

y_train_3b = np.ravel(y_train_3b)

In [ ]:
gbr = GradientBoostingRegressor(loss="absolute_error", verbose = 50)

param_grid = dict(
    learning_rate=[0.05, 0.1, 0.2],
    n_estimators = [100, 150, 200],
    max_depth=[20, 25, 30, 35],
    min_samples_leaf=[20, 30, 40],
    min_samples_split=[15, 20, 25, 30]    
)

grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',  
    cv=5,                                                
    n_jobs=-1                           
)

grid_search.fit(X_train_3b, y_train_3b)


In [ ]:
grid_search.best_params_

In [ ]:
all_models = {}

common_params = dict(
    learning_rate=0.05,
    n_estimators=200,
    max_depth=25,
    min_samples_leaf=30,
    min_samples_split=20,
)

gbr_mae = GradientBoostingRegressor(loss="absolute_error", **common_params)
all_models["mae"] = gbr_mae.fit(X_train_3b, y_train_3b)

for alpha in [0.05, 0.5, 0.95]:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
    all_models["q %1.2f" % alpha] = gbr.fit(X_train_3b, y_train_3b)

In [ ]:
X_val_3b = pd.read_csv("split_data/calib_features_preprocessed_3b.csv")
y_val_3b = pd.read_csv("split_data/calib_target_preprocessed_3b.csv")

In [ ]:
y_pred = all_models["mae"].predict(X_val_3b)
y_lower = all_models["q 0.05"].predict(X_val_3b)
y_upper = all_models["q 0.95"].predict(X_val_3b)
y_med = all_models["q 0.50"].predict(X_val_3b)

In [ ]:
predictions = pd.DataFrame()
predictions['y_true'] = y_val_3b
predictions["point prediction"] = y_pred
predictions["med"] = y_med
predictions["lower"] = y_lower
predictions["upper"] = y_upper
predictions["midpoint"] = (y_upper+y_lower)/2

predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])
predictions["abs_error_med"] = abs(predictions["med"] - predictions["y_true"])
predictions["abs_error_mid"] = abs(predictions["midpoint"] - predictions["y_true"])

predictions

In [ ]:
invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
predictions['point prediction'] = np.where(
    (predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']),
    predictions['point prediction'],
    predictions['midpoint']
)
predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])

In [ ]:
invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
MWIS, coverage = score(predictions["y_true"], predictions["lower"], predictions["upper"], alpha = .20)

MWIS

In [ ]:
predictions["abs_error"].mean()

# TabPFN test

In [ ]:
import torch
from tabpfn import TabPFNRegressor

In [ ]:
X_train = pd.read_csv("train_features_preprocessed_3b.csv")
y_train = pd.read_csv("train_target_preprocessed_3b.csv")

y_train = np.ravel(y_train)

In [ ]:
reg1 = TabPFNRegressor(n_estimators = 64, ignore_pretraining_limits=True, softmax_temperature = 0.6, average_before_softmax = True, categorical_features_indices = [0, 6, 8, 12, 13, 14])  
reg1.fit(X_train, y_train)

In [ ]:
X_val_3b = pd.read_csv("split_data/calib_features_preprocessed_3b.csv")
y_val_3b = pd.read_csv("split_data/calib_target_preprocessed_3b.csv")

In [ ]:
predictions1 = reg1.predict(X_val_3b)

In [ ]:
predictions2 = reg1.predict(X_val_3b, output_type = "quantiles", quantiles = [0.05, 0.5, 0.95])
predictions2

In [ ]:
predictions = pd.DataFrame()
predictions['y_true'] = y_val_3b
predictions["point prediction"] = predictions1
predictions["med"] = predictions2[1]
predictions["lower"] = predictions2[0]
predictions["upper"] = predictions2[2]
predictions["midpoint"] = (predictions2[0]+predictions2[1])/2

predictions["abs_error"] = abs(predictions["point prediction"] - predictions["y_true"])
predictions["abs_error_med"] = abs(predictions["med"] - predictions["y_true"])
predictions["abs_error_mid"] = abs(predictions["midpoint"] - predictions["y_true"])

predictions

In [ ]:
invalid_rows = predictions[~((predictions['point prediction'] >= predictions['lower']) & (predictions['point prediction'] <= predictions['upper']))]
display(invalid_rows)

In [ ]:
MWIS, coverage = score(predictions["y_true"], predictions["lower"], predictions["upper"], alpha = .20)

MWIS

In [ ]:
predictions["abs_error"].mean()